<h1>Using Pascal's triangle to expand binomial expressions and calculate binomial coefficients</h1>

---


by Bruno Dzikowski<br>
Cracow University of Technology, January 2021

(1) Defining the basis for symbolic computation<br>
Let us define the Basic class containing operator overloads for the symbolic computation library. Both the Symbol and Operator classes are going to inherit this class to enable basic arithmetic.<br>
The overloaded operators are "+", "-", "*" and "^".

In [69]:
class Basic():
    def __add__(self, other):
        if other == 0:
            return self
        return Add(self, other)
    def __radd__(self, other):
        if other == 0:
            return self
        return Add(other, self)
        
    def __mul__(self, other):
        if other == 1:
            return self
        return Mul(self, other)
    def __rmul__(self, other):
        if other == 1:
            return self
        return Mul(other, self)

    def __sub__(self,other):
        if other == 0:
            return self
        return Sub(self, other)
    def __rsub__(self,other):
        if other == 0:
            return self
        return Sub(other, self)

    def __pow__(self,other):
        if other == 0:
            return 1
        if other == 1:
            return self
        return Pow(self, other)
    def __rpow__(self,other):
        if other == 0:
            return 1
        if other == 1:
            return self
        return Pow(other, self)

    def __neg__(self):
        return Neg(self)

(2) The Symbol and the Operator<br>
The Symbol class is used to define algebraic variables such as "x".
The Operator class represents a two-parameter function that applies some mathematical operation (such as addition or multiplication) on symbols and numbers. Each operator also has and assigned graphical symbol (i.e "+" for addition) so the resulting expression can be easily represented in a human-readable form.


In [70]:
class Symbol(Basic):
    def __init__(self, name):
        self.name = name
    def __repr__(self):
        return self.name

class Operator(Basic):
    def __init__(self, symbol, left, right, precedence):
        self.symbol = symbol
        self.left = left
        self.right = right
        self.precedence = precedence
    def __repr__(self):
        repr = ''
        if isinstance(self.left, Operator) and self.left.precedence < self.precedence:
            repr += '({0})'
        else:
            repr += '{0}'
        repr += '{1}'
        if isinstance(self.right, Operator) and self.right.precedence < self.precedence:
            repr += '({2})'
        else:
            repr += '{2}'
        repr = repr.format(self.left, self.symbol, self.right)
        repr = repr.replace("+-", "-")
        repr = repr.replace("--", "+")
        return repr
    def simplify(self):
        return self

(3) Defining the operators<br>
Now we define basic arithmetic operators: addition, multiplication, subtraction and exponentation. 

In [71]:
class Add(Operator):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        Operator.__init__(self, '+', left, right, 0)

class Mul(Operator):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        Operator.__init__(self, '*', left, right, 1)

class Sub(Operator):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        Operator.__init__(self, '-', left, right, 0)

class Pow(Operator):
    def __init__(self, left, right):
        self.left = left
        self.right = right
        Operator.__init__(self, '^', left, right, 2)

Now let us test the basic arithmetic by defining two symbols **x** and **y**, and then asigning the result of **x + y** in a variable called **a**:


In [72]:
x = Symbol('x')
y = Symbol('y')
a = x + y
print(a)

x+y


The variable **a** now holds the addition operator whose left side argument is the symbol **x** and right side argument is the symbol **y**. It's text representation "x+y" is then printed on the screen.

In [73]:
print((x+y)*2)
print(x+y*2)

(x+y)*2
x+y*2


Examples of mathematical operations.

(4) Pascal's triangle<br>
Pascal's triangle is a triangular array of binomial coefficients consisting of n rows where each row is only as long as the row's index. It has one unique nonzero entry at row 1 of length 1. Each entry is created by adding the number above it to the left to the number above it and to the right (Entries at indexes overlfowing the array size are treated as 0). Let us define a function that generates the Pascal's triangle consisting of the chosen number of rows:


In [74]:
def pascals_triangle(rows):
    triangle = []
    for i in range(rows):
        row = []
        for j in range(i+1):
            if j == 0 or j == i:
                row.append(1)
            else:
                row.append(triangle[i-1][j-1]+triangle[i-1][j])
        triangle.append(row)
    return triangle

We can now generate a triangle of, for example, 5 rows:

In [75]:
print(pascals_triangle(5))

[[1], [1, 1], [1, 2, 1], [1, 3, 3, 1], [1, 4, 6, 4, 1]]


Let us also define the unary operator class, which in constrast to the previously defined binary operator takes only one argument. Then we can use that class to define the negation operator alowing us to use negative numbers in the future examples.


In [76]:
class Unary(Basic):
    def __init__(self, symbol, operand, precedence):
        self.symbol = symbol
        self.operand = operand
        self.precedence = precedence
    def __repr__(self):
        return '{0}{1}'.format(self.symbol, self.operand)

class Neg(Unary):
    def __init__(self, operand):
        self.operand = operand
        Unary.__init__(self, "-", operand, 0)

In [58]:
print(-x)

-x


(5) Expanding binomial expressions<br>
We can use the Pascal's triangle to expand binomial expressions (i.e (x+y)<sup>2</sup>). The following function first checks if the passed expression is a binomial, then generates Pascals triangle of size n, where n is equal to the exponent z in the expression (x+y)<sup>z</sup>. The coefficients are then extracted from the nth row of the Pascal's triangle and an expanded expression is returned.

In [77]:
def expand_binomial(binomial):
    if not isinstance(binomial, Pow):
        raise Exception
    if not (isinstance(binomial.left, Add) or isinstance(binomial.left, Sub)):
        raise Exception
    power = binomial.right
    if type(power) is not int:
        raise Exception

    x = binomial.left.left
    y = binomial.left.right
    if (isinstance(binomial.left, Sub)):
        y = -y

    expaned = 0
    t = pascal_traingle(power + 1)
    row = t[power]
    for i in range(len(row)):
        expaned += row[i] * x**(len(row)-i-1) * y**i

    return expaned


In [78]:
e = expand_binomial((x+y)**2)
print(e)

x^2+2*x*y+y^2


(6) Binomial coefficients<br>
We can also use the Pascal's triangle to calculate binomial coefficients with the following simple function:

In [79]:
def binomial(n, b):
    if b > n:
        raise Exception
    return pascal_traingle(n + 1)[n][b]

In [80]:
print(binomial(5, 2))

10


The next thing that would need to be implemented in a symbolic computation library would be simplification using term rewriting. For example we can add a function called simplify that will convert the expression x(a+b+c) to the form x\*a+x\*b+x\*c

In [81]:
def simplify(term):
    if not isinstance(term, Operator):
        return term
    if type(term) == Mul:
        if type(term.right) == Add:
            term = term.left*term.right.left + term.left*term.right.right
        if type(term.left) == Add:
            term = term.right*term.left.left + term.right*term.left.right
    term.left = simplify(term.left)
    term.right = simplify(term.right)
    return term

In [85]:
x = Symbol('x')
y = Symbol('y')
a = expand_binomial((x+y)**2)
# = expand_binomial((a+y)**3)
#s = simplify(2*a)
b = 2*expand_binomial((x+y)**2)
print(b)
print(simplify(b))

2*(x^2+2*x*y+y^2)
2*x^2+2*2*x*y+2*y^2
